In [1]:
import os
import re
import pandas as pd
from khaiii import KhaiiiApi


In [2]:

# 데이터프레임을 리스트로 변경하는 함수
def dataframe_to_list(df, column_name):
    return df[column_name].astype(str).tolist()

# 텍스트 전처리 함수
def text_preprocess(text):
    # span tag 제거
    text = re.sub(r"<span[^>]*>", "", str(text))
    text = re.sub(r"</span>", "", text)
    # br tag 삭제
    text = re.sub(r"<br[/]?>", "", text)
    # 영어, 한글, 숫자, 온점 제외 삭제
    text = re.sub(r"[^0-9가-힣A-Za-z. ]", "", text)
    # 온점을 구분으로 문장 구분
    sentences = [sen.strip() for sen in text.split('.') if sen.strip()]
    return sentences



In [40]:
# CSV 파일들이 있는 디렉토리 경로
directory ='./2014/'  #'./data/2019re/' # './data_DINK_view/' #'./data_maternity/' #'./data_gender/' # './data_DINK/'  # './20140601to20150531/'#'./data_care/'  #'./data_house_view/' #'./data_house/' # './data_care/'   #'./20140601to20150531/'


In [41]:

# 'comments_'로 시작하는 모든 CSV 파일 경로 찾기
csv_files = [file for file in os.listdir(directory) if file.startswith('comments_') and file.endswith('.csv')]

# DataFrame 초기화
total_df = pd.DataFrame()


# CSV 파일들을 읽어들여 DataFrame에 추가
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    total_df = pd.concat([total_df, df], ignore_index=True)

# 'Comment' 열과 'Likes' 열을 리스트로 변환
comment_list = dataframe_to_list(total_df, 'Comment')
likes_list = dataframe_to_list(total_df, 'Likes')


In [42]:
api = KhaiiiApi()
nng_counts_with_likes = {}  # likes 수를 고려한 NNG 단어의 개수를 저장할 딕셔너리
nng_counts_without_likes = {}  # likes 수를 고려하지 않은 NNG 단어의 개수를 저장할 딕셔너리

stop_words = ["때", "요즘", "잔망", "트", "지", "진짜", "가요계", "아이돌", "피", "영지", "거", "이", "전", "투샷"]

noun_sentence = [] #all noun in one sentence


for sentence_with_likes in zip(comment_list, likes_list):
    sentence, likes = sentence_with_likes[0], sentence_with_likes[1]
    if pd.notnull(likes) and str(likes).replace(".", "").isdigit():  # NaN 값 및 소수 확인
        sentences = text_preprocess(sentence)
        for sentence in sentences:
            
            nouns = []  #all noun in one sentence
            
            for word in api.analyze(sentence):
                for morph in word.morphs:
                    if morph.tag == 'NNG' and morph.lex not in stop_words:
                        lex = morph.lex
                        
                        nouns.append(lex)  #all noun in one sentence
                        
                        if lex in nng_counts_with_likes:
                            nng_counts_with_likes[lex] += int(float(likes))
                        else:
                            nng_counts_with_likes[lex] = int(float(likes))
                        
                        if lex in nng_counts_without_likes:
                            nng_counts_without_likes[lex] += 1
                        else:
                            nng_counts_without_likes[lex] = 1


            if len(nouns)>0: #all noun in one sentence
                noun_sentence.append([" ".join(nouns), int(float(likes))])  #all noun in one sentence
         

sorted_noun_sentence= sorted(noun_sentence, key=lambda x:(x[1],x[0]) ,reverse= True)   

#print(sorted_noun_sentence[:100])
#의미연결망을 위해noun데이터 저장
noun_df = pd.DataFrame(sorted_noun_sentence, columns=['nouns','likes']) #all noun in one sentence
noun_df.to_csv('nouns.csv', index=False, encoding="utf-8-sig") #all noun in one sentence



In [43]:
#csv파일 저장

# 내림차순으로 정렬
sorted_counts_with_likes = sorted(nng_counts_with_likes.items(), key=lambda x: x[1], reverse=True)
sorted_counts_without_likes = sorted(nng_counts_without_likes.items(), key=lambda x: x[1], reverse=True)

# 상위 50개 단어 추출
top_50_with_likes = sorted_counts_with_likes[:50]
top_50_without_likes = sorted_counts_without_likes[:50]

# 상위 50개 단어 출력
print("Top 50 Words with Likes:")
for word, count in top_50_with_likes:
    print(word, count)

print("Top 50 Words without Likes:")
for word, count in top_50_without_likes:
    print(word, count)

# 결과를 CSV 파일로 저장
df_with_likes = pd.DataFrame(top_50_with_likes, columns=['Word', 'Count'])
df_with_likes.to_csv('result_with_likes.csv', index=False, encoding="utf-8-sig")

df_without_likes = pd.DataFrame(top_50_without_likes, columns=['Word', 'Count'])
df_without_likes.to_csv('result_without_likes.csv', index=False, encoding="utf-8-sig")


Top 50 Words with Likes:
애 19418
말 18604
돈 16686
출산 14795
나라 13169
사람 10793
결혼 9114
인구 9078
생각 8770
우리나라 7759
걱정 6790
일할사람 5710
여자 5525
아이 5290
집 4975
문제 4954
국가 4817
일 4781
사회 4323
집값 3919
투표 3803
부모 3722
자식 3636
수 3595
채널 3503
혼자 3126
원인 3057
이유 2940
노예 2898
가능 2809
남자 2781
전부 2762
경제 2642
감소 2452
답 2398
이상 2302
웍 2300
능력 2278
결과 2238
정도 2127
대학 2052
애기 2044
삶 2019
자신 1995
국민 1925
미래 1897
행복 1866
자연 1861
회사 1860
소리 1811
Top 50 Words without Likes:
출산 1610
돈 1595
애 1309
사람 1155
결혼 1106
나라 765
생각 756
말 716
문제 656
아이 547
인구 460
일 444
사회 426
여자 425
국가 363
집 350
남자 309
경제 306
정부 300
자식 283
정책 278
집값 266
우리나라 263
이상 249
해결 233
세계 221
여성 216
미래 214
부모 213
행복 210
지금 209
지원 209
이유 201
세대 201
세금 200
교육 188
현실 186
전부 182
국민 174
필요 174
세상 166
시대 165
가능 165
기업 164
댓글 162
시간 161
소리 157
일자리 155
노예 153
정도 150
